# A quick .NET notebook for exploring dbatools

Just to see what works :-)

![First Power Bi](.\images\dbatools.png )
# Setting up the containers for the rest of the dbachecks notebooks

To be able to follow along with the rest of the notebooks, you will need to set up two containers. 

This notebook will enable you to do that.

You will need to have Docker installed. The image is based on the SQL Server 2019 image so you will need to have docker set to use Linux Containers. Right click on the Docker icon in the notification area and if it says "Switch to Linux Containers" click to switch and wait for Docker to restart.  

![Switch To Linux Containers](.\images\switchtolinuxcontainers.png )

You will be able to run all of the code in the notebooks by creating the folder, credential and containers in this notebook and then you can click on the play button in each code block to run the code. Note - There are a few code blocks with the results already included which should not be run. They are to show you the results of a command that cannot be run against containers (setting up configuration for domain accounts for example)

## Create the folders and the credential

The code below will create a directory called dbatools-demo in your Documents folder and save a credential file for logging into the containers. You can alter the directory created by altering the `$FolderPath`  

The code has been collapsed but you can view it using the view menu in the lab or by clicking the 3 ellipses

In [99]:
$FolderPath = $Env:USERPROFILE + '\Documents\dbatoolsdemo'

########################################################
Write-Output "Creating Directory $FolderPath"
if(Test-Path $FolderPath){
    Write-Output "Path $FolderPath exists already"
}else {
 $null =    New-Item $FolderPath -ItemType Directory
}

Write-Output "Creating Directory $FolderPath\SQL1"
if(Test-Path "$FolderPath\SQL1"){
    Write-Output "Directory SQL1 exists already"
    Get-ChildItem "$FolderPath\SQL1" -Recurse| Remove-Item -Force
}else {
$null = New-Item "$FolderPath\SQL1"-ItemType Directory
}
Write-Output "Creating File $FolderPath\SQL1\dummyfile.txt"
if(Test-Path "$FolderPath\SQL1\dummyfile.txt"){
    Write-Output "dummyfile.txt exists already"
}else {
$null =  New-Item "$FolderPath\SQL1\dummyfile.txt" -ItemType file
}

Write-Output "Creating Directory $FolderPath\SQL2"
if(Test-Path "$FolderPath\SQL2"){
    Write-Output "Directory SQL2 exists already"
    Get-ChildItem "$FolderPath\SQL2" -Recurse| Remove-Item -Force
}else {
 $null =    New-Item "$FolderPath\SQL2"-ItemType Directory
}
Write-Output "Creating File $FolderPath\SQL2\dummyfile.txt"
if(Test-Path "$FolderPath\SQL2\dummyfile.txt"){
    Write-Output "dummyfile.txt exists already"
}else {
$null =  New-Item "$FolderPath\SQL2\dummyfile.txt" -ItemType file
}

Write-Output "Creating a credential file for the containers - Please don't do this in production"

$sqladminPassword = ConvertTo-SecureString 'dbatools.IO' -AsPlainText -Force 
$cred = New-Object System.Management.Automation.PSCredential ('sqladmin', $sqladminpassword)
$Cred | Export-Clixml -Path $FolderPath\sqladmin.cred
Write-Output "Credential file created"

Write-Output "Setting the docker-compose files values"
$dockercompose = (Get-Content .\dockercompose.yml) -replace '__ReplaceME__' , $FolderPath
# $dockercompose
$dockercompose | Set-Content .\docker-compose.yml



Creating Directory C:\Users\mrrob\Documents\dbatools

Creating Directory C:\Users\mrrob\Documents\dbatools\SQL1

Creating File C:\Users\mrrob\Documents\dbatools\SQL1\dummyfile.txt

Creating Directory C:\Users\mrrob\Documents\dbatools\SQL2

Creating File C:\Users\mrrob\Documents\dbatools\SQL2\dummyfile.txt

Creating a credential file for the containers - Please don't do this in production

Credential file created

Setting the docker-compose files values

## Containers

We are going to create two SQL 2019 containers using the sqldbawithabeard\dbachecks [image from the Docker Hub](https://hub.docker.com/repository/docker/sqldbawithabeard/dbachecksdemo). 

The first time it is going to pull the image sqldbawithabeard/dbachecksdemo from the Docker Hub. If you wish to do this first, you can run the cell below



In [ ]:
docker pull sqldbawithabeard/dbachecksdemo

Now we can start the containers with the code below

In [100]:
docker-compose up -d

Creating network "dbatools-netnotebook_BeardsMagicNetwork" with the default driver

Creating dbatools-netnotebook_SQL2019-1_1 ...

Creating dbatools-netnotebook_SQL2019_1 ...

Creating dbatools-netnotebook_SQL2019_1 ... done

Creating dbatools-netnotebook_SQL2019-1_1 ... done

All being well, you wil have something that looks like


![DockerCompose](.\images\containers.png )

If you get an error you might need to add the user you have shared your drives with Docker read and execute permissions to the \Documents\ directory in your user profile   
More details https://docs.docker.com/docker-for-windows/


Now we can start exploring with dbatools :-)

If you have not installed dbatools, it can be got from the PowerShell Gallery using `Install-Module dbatools` the code below will check for the module and either install it in your user profile or update it and Import it

In [ ]:
if(Get-Module dbatools -ListAvailable){
Write-Output "Updating dbatools"
Update-Module dbatools
}else {
Write-Output "Installing dbatools in your user profile"
Install-Module dbatools -Scope CurrentUser
}
Import-Module dbatools

Now that is done, we can make a connection to our instances and see if we can connect to them.

We are going to use `Connect-DBaInstance` to do this and we will use the containers that we have created and because we need to use SQL Authentication, we will use a credential that we have saved to disk using `Export-CliXML` in one of the celss above.

It is **Important** to state that this is not a production secure solution and should not be used as a means of accessing any secure system.

There will be no output from the code below but it will have created a connection to the two containers

In [101]:
$FolderPath = $Env:USERPROFILE + '\Documents\dbachecks'
$SqlInstances = 'localhost,15592','localhost,15593'
$SqlCredential = Import-Clixml -Path $FolderPath\sqladmin.cred

$SQL1 = Connect-DbaInstance -SqlInstance $SqlInstances[0] -SqlCredential $SqlCredential
$SQL2 = Connect-DbaInstance -SqlInstance $SqlInstances[1] -SqlCredential $SqlCredential

# Databases

Lets take a look at the databases that are available on the containers


In [102]:
Get-DbaDatabase -SqlInstance $SQL1,$SQL2 -SqlCredential $SqlCredential | Format-Table

ComputerName InstanceName SqlInstance Name Status IsAccessible RecoveryModel LogReu

seWait

Status

------------ ------------ ----------- ---- ------ ------------ ------------- ------

localhost MSSQLSERVER 696e115fb099 master Normal True Simple …thing

...

localhost MSSQLSERVER a0f2f02705b4 msdb Normal True Simple 13

localhost MSSQLSERVER a0f2f02705b4 AdventureWorks2017 Normal True Simple …thing

localhost MSSQLSERVER a0f2f02705b4 WideWorldImporters Normal True Simple …thing

localhost MSSQLSERVER a0f2f02705b4 Northwind Normal True Simple …thing

localhost MSSQLSERVER a0f2f02705b4 pubs Normal True Simple …thing

We have some information about each of the databases, such as their name, status,RecoveryModel, LogReUseWait status and size by default.

If we only wanted to look at the user databases on the instance we can use 

In [90]:
Get-DbaDatabase -SqlInstance $SQL1,$SQL2 -SqlCredential $SqlCredential -ExcludeSystem | Format-Table

ComputerName InstanceName SqlInstance Name Status IsAccessible RecoveryModel LogReu

seWait

Status

------------ ------------ ----------- ---- ------ ------------ ------------- ------

localhost MSSQLSERVER fea56dcfdbd5 AdventureWorks2017 Normal True Simple …thing

...

localhost MSSQLSERVER fea56dcfdbd5 pubs Normal True Simple …thing

localhost MSSQLSERVER a86fd50a08d9 AdventureWorks2017 Normal True Simple …thing

localhost MSSQLSERVER a86fd50a08d9 WideWorldImporters Normal True Simple …thing

localhost MSSQLSERVER a86fd50a08d9 Northwind Normal True Simple …thing

localhost MSSQLSERVER a86fd50a08d9 pubs Normal True Simple …thing

If we want to get a database by name

In [91]:
Get-DbaDatabase -SqlInstance $SQL1,$SQL2 -SqlCredential $SqlCredential -Database NorthWind 

ComputerName : localhost

InstanceName : MSSQLSERVER

SqlInstance : fea56dcfdbd5

Name : Northwind

Status : Normal

...

Collation : SQL_Latin1_General_CP1_CI_AS

Owner : sqladmin

LastFullBackup : 21/12/2019 14:42:01

LastDiffBackup : 21/12/2019 14:42:00

LastLogBackup : 01/01/0001 00:00:00

# Backups

We can investigate the file system from the viewpoint of the SQL Service Account using `Get-DbaFile`

If we look in the Directory that we created earlier

In [103]:
Get-DbaFile -SqlInstance $SQL1 -SqlCredential $SqlCredential -Path /var/opt/mssql/backups/SQL1

Filename SqlInstance

-------- -----------

/var/opt/mssql/backups/SQL1\dummyfile.txt 696e115fb099

You can see there is just the dummy file we added. You should add the docker account with full control over your dbatoolsdemo directory in your User Profile directory if you get errors here

In [104]:
Backup-DbaDatabase  -SqlInstance $SQL1 -SqlCredential $SqlCredential -Path /var/opt/mssql/backups/SQL1

PROGRESS: Complete

PROGRESS: Complete

PROGRESS: Complete

PROGRESS: Complete

PROGRESS: Complete

...

696e115fb099 model Full 2.59 MB Disk 2020-02-24 18:48:43.000 00:00:00 2020-0…

696e115fb099 msdb Full 16.09 MB Disk 2020-02-24 18:48:43.000 00:00:01 2020-0…

696e115fb099 Northwind Full 6.71 MB Disk 2020-02-24 18:48:44.000 00:00:00 2020-0…

696e115fb099 pubs Full 4.46 MB Disk 2020-02-24 18:48:45.000 00:00:00 2020-0…

696e115fb099 WideWorldImporters Full 444.60 MB Disk 2020-02-24 18:48:46.000 00:00:08 2020-0…

In [105]:
Get-DbaFile -SqlInstance $SQL1 -SqlCredential $SqlCredential -Path /var/opt/mssql/backups/SQL1

Filename SqlInstance

-------- -----------

/var/opt/mssql/backups/SQL1\AdventureWorks2017_202002241848.bak 696e115fb099

/var/opt/mssql/backups/SQL1\dummyfile.txt 696e115fb099

/var/opt/mssql/backups/SQL1\master_202002241848.bak 696e115fb099

/var/opt/mssql/backups/SQL1\model_202002241848.bak 696e115fb099

/var/opt/mssql/backups/SQL1\msdb_202002241848.bak 696e115fb099

/var/opt/mssql/backups/SQL1\Northwind_202002241848.bak 696e115fb099

/var/opt/mssql/backups/SQL1\pubs_202002241848.bak 696e115fb099

/var/opt/mssql/backups/SQL1\WideWorldImporters_202002241848.bak 696e115fb099

In [47]:
Backup-DbaDatabase -SqlInstance $SQL1 -SqlCredential $SqlCredential -CreateFolder -Path /var/opt/mssql/backups/ -CompressBackup -WarningAction SilentlyContinue

PROGRESS: Complete

PROGRESS: Complete

PROGRESS: Complete

PROGRESS: Complete

PROGRESS: Complete

...

0ff1dc927365 model Full 2.59 MB Disk 2020-02-24 17:33:27.000 00:00:00 2020-0…

0ff1dc927365 msdb Full 16.09 MB Disk 2020-02-24 17:33:27.000 00:00:01 2020-0…

0ff1dc927365 Northwind Full 6.71 MB Disk 2020-02-24 17:33:28.000 00:00:00 2020-0…

0ff1dc927365 pubs Full 4.46 MB Disk 2020-02-24 17:33:29.000 00:00:00 2020-0…

0ff1dc927365 WideWorldImporters Full 444.73 MB Disk 2020-02-24 17:33:29.000 00:00:04 2020-0…

In [ ]:
$a.ForEach{Invoke-DbaQuery -SqlInstance $sql1 -SqlCredential $SqlCredential -Query $psitem }

In [ ]:
$a.GetType()

# BackUps

Lets check the last time that these databases were backed up

In [ ]:
Get-DbaLastBackup -SqlInstance $SQL1,$SQL2 -SqlCredential $SqlCredential

In [ ]:
Get-Dbalogin -SqlInstance $sql1 | Format-Table

Of course, you are not limited to just one instance with dbatools. Lets get the databases on all instances

In [46]:
Get-DbaDatabase -SqlInstance $SQL1 , $SQL2 | Format-Table

ComputerName InstanceName SqlInstance Name Status IsAccessible RecoveryModel LogReu

seWait

Status

------------ ------------ ----------- ---- ------ ------------ ------------- ------

localhost MSSQLSERVER 0ff1dc927365 master Normal True Simple …thing

...

localhost MSSQLSERVER 764069d78325 msdb Normal True Simple 13

localhost MSSQLSERVER 764069d78325 AdventureWorks2017 Normal True Simple …thing

localhost MSSQLSERVER 764069d78325 WideWorldImporters Normal True Simple …thing

localhost MSSQLSERVER 764069d78325 Northwind Normal True Simple …thing

localhost MSSQLSERVER 764069d78325 pubs Normal True Simple …thing

or maybe all of the logins

In [ ]:
Get-DbaLogin -SqlInstance $SQL1 , $SQL2 -ExcludeSystemLogin | Format-Table

SetValueInvocationException: 
Line |
 6 | $rawUI.BufferSize = $newSize

 | ^ Exception setting "BufferSize": "A command that prompts the user failed because the host program or the command type does not support user interaction. Try a host program that supports user interaction, such as the PowerShell Console, and remove prompt-related commands from command types that do not support user interaction."

Width Height

----- ------

30 100

# Clean Up

The code below will clean up the files that have been created and remove the docker containers but not the image so tha tyou can run this notebook again without having to download the image again

In [106]:
docker-compose down 
$FolderPath = $Env:USERPROFILE + '\Documents\dbatoolsdemo'
Remove-Item $FolderPath -Recurse -Force

Stopping dbatools-netnotebook_SQL2019-1_1 ...

Stopping dbatools-netnotebook_SQL2019_1 ...

Stopping dbatools-netnotebook_SQL2019_1 ... done

Stopping dbatools-netnotebook_SQL2019-1_1 ... done

Removing dbatools-netnotebook_SQL2019-1_1 ...

Removing dbatools-netnotebook_SQL2019_1 ...

Removing dbatools-netnotebook_SQL2019-1_1 ... done

Removing dbatools-netnotebook_SQL2019_1 ... done

Removing network dbatools-netnotebook_BeardsMagicNetwork